In [ ]:
"""  
copyright 6 December 2020
     Serdar AKYOL
"""

In [1]:
import pandas as pd
import numpy as np
import gensim
import re
import nltk

In [2]:
def load_dataset(path):
    df = pd.read_pickle(path)
    df = df.drop(df.columns[[1,2,4,5,3]], axis=1)
    pd.set_option('max_colwidth', None)
    return df
df = load_dataset(r'/home/ak/Desktop/clustered_test_datasets/clustered_test/clustered_test_998_group_4.pkl')
df.head(1)

,Content
14053,"Euro Bölgesi büyüme öngörüsü altı yılın en düşüğünde\nIMF Euro Bölgesi büyüme öngörüsünü altı yılın en düşük düzeyine çekti\nUluslararası Para Fonu (IMF), Bölgesel Ekonomik Görünüm raporunda Euro Bölgesi gayrisafi yurt içi hasılasının son altı yılın en düşük büyüme hızına gerileyerek, 2019 yılında yüzde 0.1 düşüşle yüzde 1.2 düzeyinde gerçekleşeceğini öngördü.\nEuro Bölgesi için 2020 ve 2021 yılına ilişkin büyüme tahminlerini yüzde 1.5'ten yüzde 1.4'e düşüren IMF, düşüşün nedeni olarak Almanya ve İtalya ekonomilerinde görülen zayıflamayı gösterdi.IMF, 19 ülkeden oluşan Euro Bölgesi'nin bu yıl için büyüme öngörüsünü, daha önceki yüzde 1.3 düzeyinden yüzde 1.2'ye revize etti. Euro Bölgesi'nin 2018 büyümesi yüzde 1.9 düzeyindeydi.\nAlmanya ve İtalya için önceki büyüme tahminlerini revize eden IMF, yavaşlamanın çoğunlukla euro bölgesinin en büyük ekonomisi Almanya'daki zayıf büyüme ve üçüncü büyük ekonomisi İtalya'daki durgunluktan kaynaklandığını ifade etti.\nIMF Almanya ekonomisindeki büyüme beklentisini Nisan ayı tahmini olan yüzde 0.8'den yüzde 0.5'e ve İtalyan ekonomisinin de duraklayacağını öngörerek büyüme beklentisini yüzde 0.9'dan yüzde 0.5'e düşürdü.\nETİKETLER :\n \n IMF\n euro bölgesi"


In [3]:
myRegex = '!"#$%&\'()*+,-/:.;<=>?@[\\]^_`{|}~'
stop_word_list = nltk.corpus.stopwords.words('turkish')

#  Clean dataset
def clean_text(text):
    text = text.lower()
    text = re.sub('\[*?\]', ' ', text)
    text = re.sub('[%s]' % re.escape(myRegex), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', ' ', text)
    text = re.sub('\t', '', text)
    text = re.sub('\n', '', text)
    word_tokens = nltk.word_tokenize(text)
    filtered_sentences = [w for w in word_tokens if not w in stop_word_list if len(w) > 1]

    text = ' '.join(filtered_sentences)
    return text

clean_first_step = lambda x: clean_text(x)
df_list = df['Content'].values.tolist()
cleaned_list = []
for news in df_list:
    cleaned_list.append(clean_text(news))

#  convert list of string to list of list
cleaned_list_of_list = [news.strip("[]").split(" ") for news in cleaned_list]

In [4]:
#  delete unnecessary variables
del df, df_list, stop_word_list, myRegex, news

In [5]:
def load_model(path):
    return gensim.models.Word2Vec.load(path)
model = load_model(r'../data/word2vec_corpus_big_v0.1/corpus_big.w2v')
#  w2v_vectors = model.wv.vectors #  load vectors for each word in the model
#  w2v_indices = {word: model.wv.vocab[word].index for word in model.wv.vocab} #  load indices for each word in the model

In [6]:
#  get average of every news
def get_average (news):
    counter = 0
    sum = np.zeros([100,], dtype=np.float32)
    for news_word in news:
        if news_word in model.wv:
            data = model[news_word]
            sum = np.add(sum, model[news_word])
            counter += 1
    return sum / len(news)

#  add every news average into a list
average_of_news_vectors = []
for news_items in cleaned_list_of_list:
    average_of_news_vectors.append(get_average(news_items))

In [7]:
def cos_similarity (firstItem, secondItem):
    coss = np.dot(firstItem, secondItem) / (np.linalg.norm(firstItem) * np.linalg.norm(secondItem))
    return coss

In [65]:
indises = []
for i in range(len(average_of_news_vectors)):
    for j in range(i + 1, len(average_of_news_vectors)):
        result = cos_similarity(average_of_news_vectors[i], average_of_news_vectors[j])
        if result >= 0.9:
            indises.append(j)

In [67]:
unique_indises = set(indises)
for i in unique_indises:
    f = open("same_news_2.txt", "a")
    f.write(cleaned_list[i])
    f.write("\n")
f.close()

In [68]:
for i in range(len(cleaned_list)):
    if i not in unique_indises:
        f = open("same_news_2.txt", "a")
        f.write(cleaned_list[i])
        f.write("\n")
f.close()